# Solución numérica del péndulo simple

*(si no tienes Python y/o Jupyter instalado localmente, considera usar Google Colab para ejecutar este cuadernillo; una cuenta Google Drive puede ser necesaria)* <a href="https://colab.research.google.com/github/djmunoz/cursos-fisica-2/blob/main/pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib notebook

from IPython import display
from numpy import sin, cos
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation
from collections import deque

# Turn off matplotlib plot in Notebook
plt.ioff()
# Pass the ffmpeg path
plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'


# El péndulo simple


In [9]:


g = 9.8  # aceleration de gravedad in m/s^2
L = 1.0  # longitud del péndulo en m
M = 1.0  # masa del péndulo en kg
t_stop = 15  # cuántos segundos para evaluar
history_len = 500  # cuántos puntos para graficar
omega_natural = np.sqrt(g/L)
omega_pert = omega_natural

def eom(y, t): # ecuación de movimiento

    theta = y[0]
    thetadot = y[1]  

    dtheta_dt = thetadot
    dthetadot_dt = -g/L * sin(theta) 
    dthetadot_dt -= 0.05 * thetadot
    dthetadot_dt += 3*np.cos(omega_pert*t)
   
    
    return [dtheta_dt,dthetadot_dt]

# crear un arreglo para el tiempo entre 0..t_stop con paso de 0.02 segundos
dt = 0.02
t = np.arange(0, t_stop, dt)

# th1 and th2 are the initial angles (degrees)
# w10 and w20 are the initial angular velocities (degrees per second)
theta0 = 20.0
omega0 = 0.0


# estado inicial
y0 = np.radians([theta0, omega0])

# integrar la EDO usando scipy.integrate.
y = integrate.odeint(eom, y0, t)

x = L*sin(y[:, 0])
y = -L*cos(y[:, 0])

fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False, xlim=(-L, L), ylim=(-L, 1.))
ax.set_xlim(-1.2,1.2)
ax.set_ylim(-1.2,0.8)
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
trace, = ax.plot([], [], ',-', lw=1)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
history_x, history_y = deque(maxlen=history_len), deque(maxlen=history_len)


def animate(i):
    thisx = [0, x[i]]
    thisy = [0, y[i]]

    if i == 0:
        history_x.clear()
        history_y.clear()

    history_x.appendleft(thisx[1])
    history_y.appendleft(thisy[1])

    line.set_data(thisx, thisy)
    trace.set_data(history_x, history_y)
    al=1
    co = (1, 0.644, 0, al)
    trace.set_color(co)

    time_text.set_text(time_template % (i*dt))
    return line, trace, time_text


ani = animation.FuncAnimation(
    fig, animate, len(y), interval=dt*1000, blit=True)

video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()
